In [ ]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

Since there is basically no external data file, this notebook is extremly simple to open in collab and to train your models there.

# **Task 4: **

**We're going to build a network that takes and converts dates from one format into another. **

For example, given a date string such as "14-03-2020", we want out network to, character by character read this string and output to us "The 14th of March 2020".

Since our data is a sequence of information, each part derives it's meaning from a prior part.
"2" as the second month character could either encode for Feb or for december depending on what number preceded it. This is a problem that is well handled by recurrent neural networks. 

We're going to be using **Transformer** to build this network, 


### Make Dataset

In [ ]:
# we will use the same dataset which we use in task 3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def make_short_date(dt):
    return dt.strftime('%d-%m-%Y')

def make_long_date(dt):
    date = dt.strftime('%d')
    if date[-1] == '1':
        suffix = 'st'
    elif date[-1] == '2':
        suffix = 'nd'
    elif date[-1] == '3':
        suffix = 'rd'
    else:
        suffix = 'th'
    month = dt.strftime('%B')
    year = dt.strftime('%Y')
    
    return date + suffix + ' of ' + month + ' ' + year

def make_dataset(n):
    dates = pd.date_range(datetime(1990, 4, 14), periods=n, normalize=True)
    
    x = dates.map(make_short_date).values
    y = dates.map(make_long_date).values
    
    return x, y

#hugging face


In [ ]:
from transformers.models.bert.modeling_bert import BertModel,BertForMaskedLM


In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
data_path="/content/drive/MyDrive/deep learning/pa6/rnn_data.txt"

In [ ]:
from transformers import BertGenerationConfig, BertGenerationEncoder,BertGenerationDecoder,EncoderDecoderModel,BertTokenizer

In [ ]:
model_checkpoint = "t5-small"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Could not locate the tokenizer configuration file, will try to use the model config instead.
https://huggingface.co/t5-small/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpi3wm1bj2


Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

storing https://huggingface.co/t5-small/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
creating metadata file for /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

storing https://huggingface.co/t5-small/resolve/main/spiece.model in cache at /root/.cache/huggingface/transformers/65fc04e21f45f61430aea0c4fedffac16a4d20d78b8e6601d8d996ebefefecd2.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d
creating metadata file for /root/.cache/huggingface/transformers/65fc04e21f45f61430aea0c4fedffac16a4d20d78b8e6601d8d996ebefefecd2.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d
https://huggingface.co/t5-small/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmprljqq6ow


Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

storing https://huggingface.co/t5-small/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/06779097c78e12f47ef67ecb728810c2ae757ee0a9efe9390c6419783d99382d.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529
creating metadata file for /root/.cache/huggingface/transformers/06779097c78e12f47ef67ecb728810c2ae757ee0a9efe9390c6419783d99382d.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529
loading file https://huggingface.co/t5-small/resolve/main/spiece.model from cache at /root/.cache/huggingface/transformers/65fc04e21f45f61430aea0c4fedffac16a4d20d78b8e6601d8d996ebefefecd2.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d
loading file https://huggingface.co/t5-small/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/06779097c78e12f47ef67ecb728810c2ae757ee0a9efe9390c6419783d99382d.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529
loading file https://huggingface.co/t5-small/reso

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

storing https://huggingface.co/t5-small/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/fee5a3a0ae379232608b6eed45d2d7a0d2966b9683728838412caccc41b4b0ed.ddacdc89ec88482db20c676f0861a336f3d0409f94748c209847b49529d73885
creating metadata file for /root/.cache/huggingface/transformers/fee5a3a0ae379232608b6eed45d2d7a0d2966b9683728838412caccc41b4b0ed.ddacdc89ec88482db20c676f0861a336f3d0409f94748c209847b49529d73885
loading weights file https://huggingface.co/t5-small/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/fee5a3a0ae379232608b6eed45d2d7a0d2966b9683728838412caccc41b4b0ed.ddacdc89ec88482db20c676f0861a336f3d0409f94748c209847b49529d73885
All model checkpoint weights were used when initializing T5ForConditionalGeneration.

All the weights of T5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use 

In [ ]:
import pandas as pd

df = pd.read_csv(
    data_path, sep='\t',header=None)
print(df)

               0                    1
0     14-04-1990   14th of April 1990
1     15-04-1990   15th of April 1990
2     16-04-1990   16th of April 1990
3     17-04-1990   17th of April 1990
4     18-04-1990   18th of April 1990
...          ...                  ...
9995  25-08-2017  25th of August 2017
9996  26-08-2017  26th of August 2017
9997  27-08-2017  27th of August 2017
9998  28-08-2017  28th of August 2017
9999  29-08-2017  29th of August 2017

[10000 rows x 2 columns]


In [ ]:
train_data=df[:8000]
test_data=df[8000:9000]
val_data=df[9000:10000]

In [ ]:
import datasets
train_data = datasets.Dataset.from_pandas(train_data)
test_data = datasets.Dataset.from_pandas(test_data)
val_data = datasets.Dataset.from_pandas(val_data)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(test_data, batch_size=8)

In [ ]:
max_input_length = 20
max_target_length = 20


def preprocess_function(examples):
    inputs = [doc for doc in examples["0"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, padding=True ,truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["1"], max_length=max_target_length,padding=True ,truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
train_data = train_data.map(preprocess_function, batched=True)
val_data = val_data.map(preprocess_function, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [ ]:
from datasets import load_metric
import pandas as pd
import datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    output_dir="./",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    
)

Using amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: 1, 0. If 1, 0 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2000


Epoch,Training Loss,Validation Loss
1,0.045900,0.003702
2,0.020500,0.001889


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
Model weights saved in ./checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./checkpoint-500/tokenizer_config.json
Special tokens file saved in ./checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./checkpoint-1000
Configuration saved in ./checkpoint-1000/config.json
Model weights saved in ./checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: 1, 0. If 1, 0 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to ./checkpoint-1500
Configuration saved in ./checkpoint-150

TrainOutput(global_step=2000, training_loss=0.30793918561935424, metrics={'train_runtime': 497.2293, 'train_samples_per_second': 32.178, 'train_steps_per_second': 4.022, 'total_flos': 29606019072000.0, 'train_loss': 0.30793918561935424, 'epoch': 2.0})

In [ ]:
def generate(text):
    inputs = text
    inputs = tokenizer([inputs], max_length=max_input_length, truncation=True, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    generated_ids =  model.generate(input_ids,
                          attention_mask=attention_mask,
                          num_return_sequences=1,
                          do_sample=False,
                          top_k= 50,
                          top_p= 0.95,)
    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    return preds

In [ ]:

model = AutoModelForSeq2SeqLM.from_pretrained('./').to("cuda")

In [ ]:
model.save_pretrained('/content/drive/MyDrive/deep learning/pa6/task4model')


Configuration saved in /content/drive/MyDrive/deep learning/pa6/task4model/config.json
Model weights saved in /content/drive/MyDrive/deep learning/pa6/task4model/pytorch_model.bin


In [ ]:
generate('19-3-1990')

['19th of March 1990']

In [ ]:
generate('15-4-2003')

['15th of April 2003']

In [ ]:
generate('03-3-2019')

['03th of March 2019']

In [ ]:
generate('15-4-1987')

['15th of April 1987']

In [ ]:
generate('15-4-2034')

['15th of April 34']

In [ ]:

generate('15-4-2134')

['15th of April 34']